This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '*******'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2019-08-01&end_date=2019-08-01&api_key={}'.format(API_KEY)
sample = requests.get(url)
print(sample.json())

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2019-08-01', 'end_date': '2019-08-01', 'frequency': 'daily', 'data': [['2019-08-01', 99.6, 100.7, 99.3, 100.5, None, 102399.0, 10267086.0, None, None, None]], 'collapse': None, 'order': None}}


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
def depth(d):
    return max(depth(v) if isinstance(v,dict) else 0 for v in d.values()) + 1

print('The JSON object is a {}-level nested dictionary.\n'.format(depth(sample.json())))

print('The length of the first level is {}.'.format(len(sample.json())))
print('The key of the first level is {}.\n'.format(sample.json().keys()))
print('The length of the second level is {}.'.format(len(sample.json()['dataset_data'])))
print('The keys of the second level are {}.'.format(sample.json()['dataset_data'].keys()))

The JSON object is a 2-level nested dictionary.

The length of the first level is 1.
The key of the first level is dict_keys(['dataset_data']).

The length of the second level is 10.
The keys of the second level are dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order']).


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### Q1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [5]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key={}'.format(API_KEY)
y2017 = requests.get(url)

### Q2. Convert the returned JSON object into a Python dictionary.

In [6]:
y2017_dict = y2017.json()

### Q3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [7]:
print('There are {} days that have missing open prices.'.format(len([v[1] for v in y2017_dict['dataset_data']['data'] if v[1] == None])))
print("There are {} days that don't have missing open prices.\n".format(len([v[1] for v in y2017_dict['dataset_data']['data'] if v[1] != None])))

opens = [v[1] for v in y2017_dict['dataset_data']['data'] if v[1] != None]
print('The highest opening price in 2017 is {}.'.format(max(opens)))
print('The lowest opening price in 2017 is {}.'.format(min(opens)))

There are 3 days that have missing open prices.
There are 252 days that don't have missing open prices.

The highest opening price in 2017 is 53.11.
The lowest opening price in 2017 is 34.0.


### Q4. What was the largest change in any one day (based on High and Low price)?

In [8]:
changes = [v[2] - v[3] for v in y2017_dict['dataset_data']['data'] if v[2] != None and v[3] != None]
print('The largest daily change (based on High and Low price) in 2017 is {}.'.format(max(changes)))

The largest daily change (based on High and Low price) in 2017 is 2.8100000000000023.


### Q5. What was the largest change between any two days (based on Closing Price)?

In [9]:
closes = [v[4] for v in y2017_dict['dataset_data']['data'] if v[4] != None]
diff1 = max(map(abs, [j-i for i, j in zip(closes[:-1], closes[1:])]))
diff2 = max([abs(x) for x in [closes[i+1] - closes[i] for i in range(len(closes)-1)]])
print('Solution1: The largest change between any two days (based on Closing Price) is', diff1, end=".\n")
print('Solution2: The largest change between any two days (based on Closing Price) is', diff2, end=".")

Solution1: The largest change between any two days (based on Closing Price) is 2.559999999999995.
Solution2: The largest change between any two days (based on Closing Price) is 2.559999999999995.

### Q6. What was the average daily trading volume during this year?

In [10]:
daily_vol = [v[6] for v in y2017_dict['dataset_data']['data'] if v[6] != None]
print('The average daily trading volume in 2017 is', sum(daily_vol)/len(daily_vol), end='.')

The average daily trading volume in 2017 is 89124.33725490196.

### Q7. What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [11]:
def bubbleSort(lst):
    n = len(lst)
    for i in range(n-1):
        for j in range(0, n-i-1):
            if lst[j] > lst[j+1]:
                lst[j], lst[j+1] = lst[j+1], lst[j]

def custom_median(lst):
    n = len(lst)
    bubbleSort(lst)
    if n % 2 == 0:
        m = (lst[n//2] + lst[n//2 - 1])/2
    else:
        m = lst[n//2]
    return m

print('The median trading volume in 2017 is', custom_median(daily_vol), end='.\n')

import statistics
print('Verification using median function in statistics library: The median trading volume in 2017 is', statistics.median(daily_vol), end='.')

The median trading volume in 2017 is 76286.0.
Verification using median function in statistics library: The median trading volume in 2017 is 76286.0.